In [1]:
"""
BANGLA SER: CNN+LSTM for Spectrograms, Deep DNN with Feature Gating for Static Features
Fixed architecture - no fake temporal sequences for 1D features
Enhanced with Feature Gating mechanism for better feature selection with limited data
"""

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
import librosa
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm import tqdm
import pickle
import warnings
import math
warnings.filterwarnings('ignore')

# Set seeds
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.backends.cudnn.benchmark = True

# ==================== Feature Preparation ====================

def prepare_features_from_cache(cache_path='comprehensive_features.pkl'):
    """Load and prepare different feature sets from cache"""
    
    print(f"Loading features from {cache_path}")
    with open(cache_path, 'rb') as f:
        data = pickle.load(f)
    
    features = {}
    
    # 1. Mel-spectrogram (3-channel: original + delta + delta-delta)
    if 'mel_3channel' in data:
        features['mel_spectrogram'] = data['mel_3channel']
        print(f"Mel-spectrogram shape: {features['mel_spectrogram'].shape}")
    
    # 2. MFCCs and other acoustic features
    if 'acoustic_features' in data and 'feature_names' in data:
        acoustic_features = data['acoustic_features']
        feature_names = data['feature_names']
        
        # Extract MFCC-related features
        mfcc_indices = [i for i, name in enumerate(feature_names) if 'mfcc' in name.lower()]
        if mfcc_indices:
            features['mfcc'] = acoustic_features[:, mfcc_indices]
            features['mfcc_names'] = [feature_names[i] for i in mfcc_indices]
            print(f"MFCC features shape: {features['mfcc'].shape}")
        
        # 3. Handcrafted Bangla features (all non-MFCC acoustic features)
        bangla_indices = [i for i, name in enumerate(feature_names) 
                         if 'mfcc' not in name.lower()]
        features['bangla_handcrafted'] = acoustic_features[:, bangla_indices]
        features['bangla_names'] = [feature_names[i] for i in bangla_indices]
        print(f"Bangla handcrafted features shape: {features['bangla_handcrafted'].shape}")
        
        # 4. All acoustic features combined
        features['all_acoustic'] = acoustic_features
        features['all_names'] = feature_names
        print(f"All acoustic features shape: {features['all_acoustic'].shape}")
    
    # Labels
    if 'labels' in data:
        features['labels'] = data['labels']
    else:
        raise ValueError("Could not find labels in the cached data")
    
    features['emotion_str'] = data.get('emotion_str', None)
    
    if not isinstance(features['labels'], np.ndarray):
        features['labels'] = np.array(features['labels'])
    
    # Detect number of classes automatically
    unique_labels = np.unique(features['labels'])
    features['num_classes'] = len(unique_labels)
    features['unique_labels'] = unique_labels
    
    print(f"Total samples: {len(features['labels'])}")
    print(f"Number of emotion classes: {features['num_classes']}")
    print(f"Unique emotion labels: {features['unique_labels']}")
    
    # Try to get emotion names if available
    if 'emotion_names' in data:
        features['emotion_names'] = data['emotion_names']
    elif features['emotion_str'] is not None:
        # Extract unique emotion names from emotion_str
        unique_emotions = np.unique(features['emotion_str'])
        features['emotion_names'] = list(unique_emotions)
    else:
        # Generate generic names based on label numbers
        features['emotion_names'] = [f'Emotion_{i}' for i in unique_labels]
    
    print(f"Emotion names: {features['emotion_names'][:10]}...")  # Show first 10
    
    return features

"""
Adaptive HybridFusion Architecture for Speech Emotion Recognition
==================================================================

Combines:
1. CNN+LSTM for mel-spectrograms (temporal features)
2. Novel HybridFusion for 1D features (MFCC + acoustic features)

Feature Configurations Supported:
- MFCC only: 80 dimensions
- Handcrafted only: 51 dimensions  
- Combined: 131 dimensions (80 MFCC + 51 handcrafted)

Key Innovations for 1D features:
- Feature-Aware Dual-Stream Processing (Spectral vs Prosodic)
- Cross-Modal Feature Attention between feature types
- Selective KAN Application (only where complexity needed)
- Emotion-Specific Gating for adaptive routing
"""

# ==================== Lightweight KAN Component ====================

class LightweightKAN(nn.Module):
    """
    Simplified KAN layer optimized for efficiency
    Uses fewer B-spline control points and shared parameters
    """
    def __init__(self, in_features, out_features, grid_size=3, share_splines=True):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.share_splines = share_splines
        
        if share_splines:
            # Share spline parameters across input dimensions for efficiency
            self.spline_weight = nn.Parameter(torch.randn(1, out_features, grid_size) * 0.1)
        else:
            self.spline_weight = nn.Parameter(torch.randn(in_features, out_features, grid_size) * 0.1)
        
        # Efficient linear transformation
        self.linear = nn.Linear(in_features, out_features, bias=False)
        self.scale = nn.Parameter(torch.ones(1, out_features) * 0.5)
        
    def forward(self, x):
        # Linear component (efficient)
        linear_out = self.linear(x)
        
        # Simplified B-spline activation (lightweight)
        x_norm = torch.tanh(x.mean(dim=-1, keepdim=True))  # Reduce computation
        spline_act = torch.sigmoid(x_norm) * self.scale
        
        # Combine
        return linear_out * (1 + spline_act)


# ==================== Feature-Specific Processing Blocks ====================

class SpectralProcessingBlock(nn.Module):
    """
    Optimized for MFCC/Spectral features
    Uses efficient DNN with structured sparsity
    """
    def __init__(self, in_features, out_features, dropout=0.2):
        super().__init__()
        
        # Efficient factorized layers (reduces parameters)
        mid_features = int(math.sqrt(in_features * out_features))
        
        self.process = nn.Sequential(
            nn.Linear(in_features, mid_features),
            nn.BatchNorm1d(mid_features),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(mid_features, out_features),
            nn.BatchNorm1d(out_features)
        )
        
        # Residual with dimension matching
        self.residual = nn.Linear(in_features, out_features) if in_features != out_features else nn.Identity()
        
    def forward(self, x):
        return F.gelu(self.process(x) + self.residual(x))


class ProsodicProcessingBlock(nn.Module):
    """
    Optimized for acoustic/prosodic features
    Uses lightweight KAN for complex non-linear patterns
    """
    def __init__(self, in_features, out_features, dropout=0.2):
        super().__init__()
        
        self.kan = LightweightKAN(in_features, out_features, grid_size=3)
        self.norm = nn.BatchNorm1d(out_features)
        self.dropout = nn.Dropout(dropout)
        
        # Residual with dimension matching
        self.residual = nn.Linear(in_features, out_features) if in_features != out_features else nn.Identity()
        
    def forward(self, x):
        out = self.kan(x)
        out = self.norm(out)
        out = self.dropout(out)
        return F.gelu(out + self.residual(x))


# ==================== Novel Cross-Modal Attention ====================

class CrossModalFeatureAttention(nn.Module):
    """
    Learns relationships between different feature types
    Lightweight cross-attention mechanism
    """
    def __init__(self, dim, num_heads=4):
        super().__init__()
        self.num_heads = num_heads
        self.dim = dim
        self.head_dim = dim // num_heads
        
        # Lightweight projections
        self.q_proj = nn.Linear(dim, dim // 2)
        self.k_proj = nn.Linear(dim, dim // 2)
        self.v_proj = nn.Linear(dim, dim)
        self.out_proj = nn.Linear(dim, dim)
        
        self.scale = 1.0 / math.sqrt(self.head_dim)
        
    def forward(self, spectral_features, prosodic_features):
        B = spectral_features.shape[0]
        
        # Cross-attention: spectral queries, prosodic keys/values
        Q = self.q_proj(spectral_features).view(B, 1, self.num_heads, -1).transpose(1, 2)
        K = self.k_proj(prosodic_features).view(B, 1, self.num_heads, -1).transpose(1, 2)
        V = self.v_proj(prosodic_features).view(B, 1, self.num_heads, -1).transpose(1, 2)
        
        # Efficient attention computation
        attn = torch.matmul(Q, K.transpose(-2, -1)) * self.scale
        attn = F.softmax(attn, dim=-1)
        
        out = torch.matmul(attn, V)
        out = out.transpose(1, 2).contiguous().view(B, -1)
        
        return self.out_proj(out)


# ==================== Emotion-Specific Gating ====================

class EmotionSpecificGating(nn.Module):
    """
    Learns to route features based on emotion-specific patterns
    Different emotions use different feature combinations
    """
    def __init__(self, feature_dim, num_emotions=7):
        super().__init__()
        
        # Emotion-specific gates for each feature stream
        self.emotion_router = nn.Sequential(
            nn.Linear(feature_dim * 2, feature_dim),
            nn.ReLU(),
            nn.Linear(feature_dim, num_emotions * 2),  # 2 gates per emotion
            nn.Sigmoid()
        )
        
        self.num_emotions = num_emotions
        
    def forward(self, spectral, prosodic, emotion_logits=None):
        # Concatenate features for routing decision
        combined = torch.cat([spectral, prosodic], dim=-1)
        gates = self.emotion_router(combined)
        
        # Reshape gates: [batch, num_emotions, 2]
        gates = gates.view(-1, self.num_emotions, 2)
        
        if emotion_logits is not None:
            # During training, use soft emotion probabilities
            emotion_probs = F.softmax(emotion_logits, dim=-1)
            gates = torch.sum(gates * emotion_probs.unsqueeze(-1), dim=1)
        else:
            # During inference, use mean gating
            gates = gates.mean(dim=1)
        
        # Apply gates
        gated_spectral = spectral * gates[:, 0:1]
        gated_prosodic = prosodic * gates[:, 1:2]
        
        return gated_spectral, gated_prosodic


# ==================== HybridFusion Module for 1D Features ====================

class HybridFusionModule(nn.Module):
    """
    Novel dual-stream processing for 1D features
    Handles MFCC (80D), Handcrafted (51D), or Combined (131D)
    """
    def __init__(self, feature_dim, hidden_dim=128, num_classes=7, dropout=0.2,
                 feature_type='auto'):
        super().__init__()
        
        # Determine feature split based on input dimension
        self.feature_type = feature_type
        
        if feature_type == 'auto':
            if feature_dim == 131:  # Combined: 80 MFCC + 51 handcrafted
                self.mfcc_dim = 80
                self.acoustic_dim = 51
                self.split_mode = 'combined'
            elif feature_dim == 80:  # Just MFCC
                self.mfcc_dim = 80
                self.acoustic_dim = 0
                self.split_mode = 'mfcc_only'
            elif feature_dim == 51:  # Just handcrafted
                self.mfcc_dim = 0
                self.acoustic_dim = 51
                self.split_mode = 'acoustic_only'
            else:
                # General case: heuristic split
                self.mfcc_dim = feature_dim // 2
                self.acoustic_dim = feature_dim - self.mfcc_dim
                self.split_mode = 'heuristic'
        elif feature_type == 'mfcc':
            self.mfcc_dim = feature_dim
            self.acoustic_dim = 0
            self.split_mode = 'mfcc_only'
        elif feature_type == 'bangla_handcrafted':
            self.mfcc_dim = 0
            self.acoustic_dim = feature_dim
            self.split_mode = 'acoustic_only'
        elif feature_type == 'all_acoustic':
            # For combined features
            self.mfcc_dim = 80
            self.acoustic_dim = feature_dim - 80
            self.split_mode = 'combined'
        
        print(f"HybridFusion configured: Total={feature_dim}, MFCC={self.mfcc_dim}, Acoustic={self.acoustic_dim}, Mode={self.split_mode}")
        
        # Build architecture based on split mode
        if self.split_mode == 'combined':
            # Both MFCC and acoustic features
            self.spectral_input = nn.Sequential(
                nn.Linear(self.mfcc_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU()
            )
            
            self.prosodic_input = nn.Sequential(
                nn.Linear(self.acoustic_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU()
            )
            
            # Dual-stream processing
            self.spectral_stream = nn.ModuleList([
                SpectralProcessingBlock(hidden_dim, hidden_dim, dropout),
                SpectralProcessingBlock(hidden_dim, hidden_dim * 2, dropout),
            ])
            
            self.prosodic_stream = nn.ModuleList([
                ProsodicProcessingBlock(hidden_dim, hidden_dim, dropout),
                ProsodicProcessingBlock(hidden_dim, hidden_dim * 2, dropout),
            ])
            
            # Cross-modal attention
            self.cross_attention_early = CrossModalFeatureAttention(hidden_dim, num_heads=4)
            self.cross_attention_late = CrossModalFeatureAttention(hidden_dim * 2, num_heads=8)
            
            # Emotion-specific gating
            self.emotion_gate = EmotionSpecificGating(hidden_dim * 2, num_classes)
            
            fusion_input_dim = hidden_dim * 4
            
        elif self.split_mode == 'mfcc_only':
            # Only MFCC features - use spectral processing
            self.feature_input = nn.Sequential(
                nn.Linear(self.mfcc_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU()
            )
            
            self.feature_stream = nn.ModuleList([
                SpectralProcessingBlock(hidden_dim, hidden_dim, dropout),
                SpectralProcessingBlock(hidden_dim, hidden_dim * 2, dropout),
                SpectralProcessingBlock(hidden_dim * 2, hidden_dim * 2, dropout),
            ])
            
            fusion_input_dim = hidden_dim * 2
            
        elif self.split_mode == 'acoustic_only':
            # Only acoustic features - use prosodic processing
            self.feature_input = nn.Sequential(
                nn.Linear(self.acoustic_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU()
            )
            
            self.feature_stream = nn.ModuleList([
                ProsodicProcessingBlock(hidden_dim, hidden_dim, dropout),
                ProsodicProcessingBlock(hidden_dim, hidden_dim * 2, dropout),
                ProsodicProcessingBlock(hidden_dim * 2, hidden_dim * 2, dropout),
            ])
            
            fusion_input_dim = hidden_dim * 2
        
        else:  # heuristic split
            # Similar to combined but with automatic dimensions
            self.spectral_input = nn.Sequential(
                nn.Linear(self.mfcc_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU()
            )
            
            self.prosodic_input = nn.Sequential(
                nn.Linear(self.acoustic_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU()
            )
            
            self.spectral_stream = nn.ModuleList([
                SpectralProcessingBlock(hidden_dim, hidden_dim, dropout),
                SpectralProcessingBlock(hidden_dim, hidden_dim * 2, dropout),
            ])
            
            self.prosodic_stream = nn.ModuleList([
                ProsodicProcessingBlock(hidden_dim, hidden_dim, dropout),
                ProsodicProcessingBlock(hidden_dim, hidden_dim * 2, dropout),
            ])
            
            self.cross_attention_early = CrossModalFeatureAttention(hidden_dim, num_heads=4)
            self.cross_attention_late = CrossModalFeatureAttention(hidden_dim * 2, num_heads=8)
            self.emotion_gate = EmotionSpecificGating(hidden_dim * 2, num_classes)
            
            fusion_input_dim = hidden_dim * 4
        
        # Feature fusion layer
        self.fusion = nn.Sequential(
            nn.Linear(fusion_input_dim, hidden_dim * 3),
            nn.BatchNorm1d(hidden_dim * 3),
            nn.ReLU(),
            nn.Dropout(dropout * 2),
            nn.Linear(hidden_dim * 3, 512),  # Output 512 features
            nn.BatchNorm1d(512),
            nn.ReLU()
        )
        
        # Store gate values for interpretability
        self.last_gate_values = None
        
    def forward(self, x, pre_emotion_logits=None):
        """
        Args:
            x: [batch, feature_dim] - input features
            pre_emotion_logits: Optional emotion predictions for gating
        """
        batch_size = x.shape[0]
        
        if self.split_mode == 'combined' or self.split_mode == 'heuristic':
            # Split features
            mfcc_features = x[:, :self.mfcc_dim]
            acoustic_features = x[:, self.mfcc_dim:self.mfcc_dim + self.acoustic_dim]
            
            # Initial projections
            spectral = self.spectral_input(mfcc_features)
            prosodic = self.prosodic_input(acoustic_features)
            
            # Layer 1 processing
            spectral = self.spectral_stream[0](spectral)
            prosodic = self.prosodic_stream[0](prosodic)
            
            # Early cross-modal attention
            cross_early = self.cross_attention_early(spectral, prosodic)
            spectral = spectral + 0.5 * cross_early
            
            # Layer 2 processing
            spectral = self.spectral_stream[1](spectral)
            prosodic = self.prosodic_stream[1](prosodic)
            
            # Late cross-modal attention
            cross_late = self.cross_attention_late(spectral, prosodic)
            prosodic = prosodic + 0.5 * cross_late
            
            # Emotion-specific gating
            gated_spectral, gated_prosodic = self.emotion_gate(
                spectral, prosodic, pre_emotion_logits
            )
            
            # Store gate information
            self.last_gate_values = {
                'spectral_gate': gated_spectral.mean().item(),
                'prosodic_gate': gated_prosodic.mean().item()
            }
            
            # Final fusion
            fused = torch.cat([gated_spectral, gated_prosodic], dim=-1)
            
        else:  # mfcc_only or acoustic_only
            # Single stream processing
            features = self.feature_input(x)
            
            for layer in self.feature_stream:
                features = layer(features)
            
            fused = features
            
            self.last_gate_values = {
                'single_stream': self.split_mode
            }
        
        output = self.fusion(fused)
        return output


# ==================== Main Adaptive Architecture ====================

class AdaptiveEmotionModel(nn.Module):
    """
    Adaptive architecture that uses appropriate processing for each feature type:
    - CNN+LSTM for spectrograms (have real temporal structure)  
    - Novel HybridFusion for static features
    
    Supports:
    - MFCC features: 80 dimensions
    - Bangla handcrafted: 51 dimensions
    - Combined: 131 dimensions (80 + 51)
    """
    
    def __init__(self, input_shape, num_classes=7, dropout_rate=0.3):
        super().__init__()
        
        self.input_type = self._determine_input_type(input_shape)
        self.num_classes = num_classes
        
        if self.input_type == 'mel_3d':  # Mel-spectrogram has real temporal structure
            in_channels = input_shape[0]
            
            # CNN layers to extract temporal features
            self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=(3, 3), padding=1)
            self.bn1 = nn.BatchNorm2d(32)
            self.pool1 = nn.MaxPool2d((2, 1))  # Pool frequency, preserve time
            
            self.conv2 = nn.Conv2d(32, 64, kernel_size=(3, 3), padding=1)
            self.bn2 = nn.BatchNorm2d(64)
            self.pool2 = nn.MaxPool2d((2, 1))
            
            self.conv3 = nn.Conv2d(64, 128, kernel_size=(3, 3), padding=1)
            self.bn3 = nn.BatchNorm2d(128)
            self.pool3 = nn.MaxPool2d((2, 1))
            
            self.conv4 = nn.Conv2d(128, 256, kernel_size=(3, 3), padding=1)
            self.bn4 = nn.BatchNorm2d(256)
            # Adaptive pooling to handle different time dimensions
            self.pool4 = nn.AdaptiveAvgPool2d((8, None))  # Keep temporal dimension
            
            # LSTM for temporal modeling (only for spectrograms!)
            lstm_input_size = 256 * 8
            self.lstm = nn.LSTM(
                lstm_input_size, 256, 
                num_layers=2, batch_first=True, 
                dropout=dropout_rate, bidirectional=True
            )
            
            # Attention for LSTM output
            self.attention = nn.Sequential(
                nn.Linear(512, 128),
                nn.Tanh(),
                nn.Linear(128, 1)
            )
            
            final_feature_size = 512
            
        elif self.input_type == 'features_1d':  # Static features - Novel HybridFusion!
            self.feature_dim = input_shape[0]
            
            # Determine feature type
            if self.feature_dim == 80:
                feature_type = 'mfcc'
            elif self.feature_dim == 51:
                feature_type = 'bangla_handcrafted'
            elif self.feature_dim == 131:
                feature_type = 'all_acoustic'
            else:
                feature_type = 'auto'
            
            # Use HybridFusion module
            self.feature_net = HybridFusionModule(
                feature_dim=self.feature_dim,
                hidden_dim=128,  # Lightweight
                num_classes=num_classes,
                dropout=dropout_rate,
                feature_type=feature_type
            )
            
            # Store module reference for gate value access
            self.last_gate_values = None
            
            final_feature_size = 512
        
        # Shared classification head
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(final_feature_size, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )
        
        # Initialize weights
        self.apply(self._init_weights)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
                
    def _determine_input_type(self, input_shape):
        """Determine if input is mel-spectrogram or 1D features"""
        if len(input_shape) == 3:  # (channels, freq, time)
            return 'mel_3d'
        else:  # (features,)
            return 'features_1d'
    
    def forward(self, x):
        if self.input_type == 'mel_3d':
            # CNN processing for spectrograms
            x = F.relu(self.bn1(self.conv1(x)))
            x = self.pool1(x)
            
            x = F.relu(self.bn2(self.conv2(x)))
            x = self.pool2(x)
            
            x = F.relu(self.bn3(self.conv3(x)))
            x = self.pool3(x)
            
            x = F.relu(self.bn4(self.conv4(x)))
            x = self.pool4(x)
            
            # Reshape for LSTM: (batch, time, features)
            batch_size = x.size(0)
            x = x.permute(0, 3, 1, 2)  # (batch, time, channels, freq)
            x = x.reshape(batch_size, x.size(1), -1)  # (batch, time, features)
            
            # LSTM processing
            lstm_out, _ = self.lstm(x)
            
            # Attention over time steps
            attention_scores = self.attention(lstm_out)
            attention_weights = F.softmax(attention_scores, dim=1)
            attended = torch.sum(lstm_out * attention_weights, dim=1)
            
            features = attended
            
        elif self.input_type == 'features_1d':
            # Get initial emotion predictions for gating (optional)
            with torch.no_grad():
                # Quick forward pass for emotion hints
                quick_features = F.adaptive_avg_pool1d(x.unsqueeze(1), 128).squeeze(1)
                pre_emotion_logits = self.classifier(
                    F.pad(quick_features, (0, 512 - 128))
                ).detach()
            
            # Process through HybridFusion
            features = self.feature_net(x, pre_emotion_logits)
            
            # Store gate values for analysis
            self.last_gate_values = self.feature_net.last_gate_values
        
        # Shared classification
        output = self.classifier(features)
        
        return output


# ==================== Adaptive Training Configuration ====================

def get_training_config(feature_type):
    """Get optimized training configuration for each feature type"""
   
    configs = {
        'mel_spectrogram': {
            'learning_rate': 5e-4,
            'weight_decay': 0.01,
            'dropout_rate': 0.3,
            'batch_size': 16,
            'epochs': 150,
            'patience': 50,
            'scheduler_patience': 7,
            'label_smoothing': 0.0,
            'gradient_clip': 1.0,
            'architecture': 'CNN+LSTM',
            'description': 'CNN+LSTM for temporal mel-spectrograms'
        },
        
        'mfcc': {
            'learning_rate': 5e-4,
            'weight_decay': 0.015,
            'dropout_rate': 0.3,
            'batch_size': 16,      
            'epochs': 200,
            'patience': 50,
            'scheduler_patience': 8,
            'label_smoothing': 0.0,
            'gradient_clip': 1.0,
            'warmup_epochs': 5,
            'architecture': 'HybridFusion',
            'description': 'HybridFusion with spectral stream processing'
        },
        
        'bangla_handcrafted': {
            'learning_rate': 5e-4,
            'weight_decay': 0.015,
            'dropout_rate': 0.3,
            'batch_size': 16,
            'epochs': 200,
            'patience': 50,
            'scheduler_patience': 8,
            'label_smoothing': 0.0,
            'gradient_clip': 1.0,
            'warmup_epochs': 5,
            'architecture': 'HybridFusion',
            'description': 'HybridFusion with prosodic stream processing'
        },
        
        'all_acoustic': {
            'learning_rate': 5e-4,
            'weight_decay': 0.015,
            'dropout_rate': 0.3,
            'batch_size': 16,
            'epochs': 200,
            'patience': 50,
            'scheduler_patience': 8,
            'label_smoothing': 0.0,
            'gradient_clip': 1.0,
            'warmup_epochs': 5,
            'architecture': 'HybridFusion',
            'description': 'HybridFusion with dual-stream KAN-DNN processing'
        }
    }
    
    return configs.get(feature_type, configs['bangla_handcrafted'])

# ==================== Training Function ====================

def train_adaptive_model(model, train_loader, test_loader, device, feature_type, epochs=50):
    """Train model with adaptive configuration"""
    
    config = get_training_config(feature_type)
    
    print(f"\nTraining Configuration:")
    print(f"  Architecture: {config['architecture']}")
    print(f"  Learning Rate: {config['learning_rate']}")
    print(f"  Batch Size: {config['batch_size']}")
    print(f"  Description: {config['description']}")
    
    model = model.to(device)
    
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss(label_smoothing=config['label_smoothing'])
    optimizer = torch.optim.AdamW(
        model.parameters(), 
        lr=config['learning_rate'], 
        weight_decay=config['weight_decay']
    )
    
    # Scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, 
        patience=config['scheduler_patience'], min_lr=1e-7
    )
    
    best_test_acc = 0
    best_test_f1 = 0
    best_epoch = 0
    best_model_state = None
    
    highest_test_acc = 0
    highest_test_f1 = 0
    highest_epoch = 0
    
    patience_counter = 0
    
    history = {
        'train_loss': [], 'train_acc': [],
        'test_acc': [], 'test_f1': []
    }
    
    for epoch in range(min(config['epochs'], epochs)):
        # Training
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0
        
        pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}')
        for inputs, labels in pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=config['gradient_clip'])
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()
            
            pbar.set_postfix({
                'loss': loss.item(),
                'acc': 100. * train_correct / train_total
            })
        
        train_acc = 100. * train_correct / train_total
        avg_train_loss = train_loss / len(train_loader)
        
        # Test evaluation
        model.eval()
        test_preds = []
        test_labels = []
        
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = outputs.max(1)
                
                test_preds.extend(predicted.cpu().numpy())
                test_labels.extend(labels.cpu().numpy())
        
        test_acc = accuracy_score(test_labels, test_preds) * 100
        test_f1 = f1_score(test_labels, test_preds, average='weighted')
        
        scheduler.step(test_acc)
        
        # Store history
        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(train_acc)
        history['test_acc'].append(test_acc)
        history['test_f1'].append(test_f1)
        
        # Track highest
        if test_acc > highest_test_acc:
            highest_test_acc = test_acc
            highest_test_f1 = test_f1
            highest_epoch = epoch + 1
        
        print(f'Epoch {epoch+1}: Train Loss={avg_train_loss:.4f}, '
              f'Train Acc={train_acc:.2f}%, Test Acc={test_acc:.2f}%, Test F1={test_f1:.4f}')
        
        # Save best model
        if test_acc > best_test_acc:
            best_test_acc = test_acc
            best_test_f1 = test_f1
            best_epoch = epoch + 1
            best_model_state = model.state_dict().copy()
            patience_counter = 0
        else:
            patience_counter += 1
        
        # Early stopping
        if patience_counter >= config['patience']:
            print(f"Early stopping at epoch {epoch+1}")
            break
    
    # Load best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    print(f"\nTraining Summary:")
    print(f"  Best Model: Epoch {best_epoch}, Acc={best_test_acc:.2f}%, F1={best_test_f1:.4f}")
    print(f"  Highest: Epoch {highest_epoch}, Acc={highest_test_acc:.2f}%, F1={highest_test_f1:.4f}")
    
    return model, best_test_acc, best_test_f1, highest_test_acc, highest_test_f1, highest_epoch, history

# ==================== Feature Importance Analysis ====================

def analyze_feature_importance(model, test_loader, feature_names, device):
    """Analyze which features are most important based on gate values"""
    
    if not hasattr(model, 'feature_gate'):
        print("Model doesn't have feature gating mechanism")
        return None
    
    model.eval()
    all_gates = []
    
    with torch.no_grad():
        for inputs, _ in test_loader:
            inputs = inputs.to(device)
            _ = model(inputs)  # Forward pass to get gate values
            if model.last_gate_values is not None:
                all_gates.append(model.last_gate_values.numpy())
    
    if not all_gates:
        return None
    
    # Average gate values across all samples
    avg_gates = np.concatenate(all_gates, axis=0).mean(axis=0)
    
    # Create importance ranking
    importance_ranking = sorted(
        [(name, gate_val) for name, gate_val in zip(feature_names, avg_gates)],
        key=lambda x: x[1],
        reverse=True
    )
    
    return importance_ranking

def visualize_feature_importance(results):
    """Visualize feature importance from gating mechanism"""
    
    # Check which experiments have feature importance
    experiments_with_gating = [exp for exp in results if 'feature_importance' in results[exp]]
    
    if not experiments_with_gating:
        print("No experiments with feature gating to visualize")
        return
    
    fig, axes = plt.subplots(1, len(experiments_with_gating), figsize=(7*len(experiments_with_gating), 6))
    if len(experiments_with_gating) == 1:
        axes = [axes]
    
    for idx, exp_name in enumerate(experiments_with_gating):
        importance = results[exp_name]['feature_importance'][:20]  # Top 20
        
        feature_names = [feat[0][:20] for feat in importance]  # Truncate long names
        gate_values = [feat[1] for feat in importance]
        
        ax = axes[idx]
        bars = ax.barh(range(len(feature_names)), gate_values, color='steelblue')
        ax.set_yticks(range(len(feature_names)))
        ax.set_yticklabels(feature_names)
        ax.set_xlabel('Gate Value (Importance)', fontsize=12)
        ax.set_title(f'{exp_name}\nTop 20 Important Features', fontsize=14, fontweight='bold')
        ax.set_xlim([0, 1])
        ax.grid(True, alpha=0.3)
        
        # Color code by importance level
        for i, (bar, val) in enumerate(zip(bars, gate_values)):
            if val > 0.7:
                bar.set_color('darkgreen')
            elif val > 0.5:
                bar.set_color('steelblue')
            else:
                bar.set_color('lightgray')
    
    plt.suptitle('Feature Importance Analysis (from Gating Mechanism)', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig('feature_importance_gating.png', dpi=150, bbox_inches='tight')
    plt.show()

# ==================== Experiment Runner ====================

def run_experiments(features_dict, device='cuda', dataset_name='SUBESCO'):
    """Run experiments with adaptive architectures"""
    
    original_labels = features_dict['labels'].copy()
    num_classes = features_dict['num_classes']
    emotion_names = features_dict.get('emotion_names', [f'Emotion_{i}' for i in range(num_classes)])
    
    print(f"\nDataset: {dataset_name}")
    print(f"Number of classes: {num_classes}")
    
    experiments = [
        ('All Features', 'all_acoustic'),
        ('Bangla Handcrafted', 'bangla_handcrafted'),
        ('MFCC', 'mfcc'),
        ('Mel-Spectrogram', 'mel_spectrogram'),
    ]
    
    results = {}
    
    for exp_name, feature_key in experiments:
        labels = original_labels.copy()
        
        if feature_key not in features_dict:
            print(f"Skipping {exp_name} - features not found")
            continue
        
        print("\n" + "="*60)
        print(f"Experiment: {exp_name}")
        print("="*60)
        
        # Get features
        features = features_dict[feature_key]
        print(f"Features shape: {features.shape}")
        
        # Determine input shape and architecture
        if feature_key == 'mel_spectrogram':
            X = features
            input_shape = features.shape[1:]
            print("Architecture: CNN+LSTM (temporal structure)")
        else:
            X = features
            input_shape = (features.shape[1],)
            print("Architecture: Deep DNN with Feature Gating")
        
        # Normalize features
        if len(X.shape) == 4:  # Mel-spectrogram
            X_normalized = np.zeros_like(X)
            for i in range(X.shape[1]):
                channel = X[:, i].reshape(X.shape[0], -1)
                scaler = StandardScaler()
                normalized = scaler.fit_transform(channel)
                X_normalized[:, i] = normalized.reshape(X.shape[0], X.shape[2], X.shape[3])
            X = X_normalized
        else:  # 1D features
            scaler = StandardScaler()
            X = scaler.fit_transform(X)
        
        # Split data (80/20)
        X_train, X_test, y_train, y_test = train_test_split(
            X, labels, test_size=0.2, stratify=labels, random_state=42
        )
        
        print(f"Split: Train={len(y_train)}, Test={len(y_test)}")
        
        # Convert to tensors
        X_train = torch.FloatTensor(X_train)
        X_test = torch.FloatTensor(X_test)
        y_train = torch.LongTensor(y_train)
        y_test = torch.LongTensor(y_test)
        
        # Get config and create dataloaders
        config = get_training_config(feature_key)
        
        train_dataset = TensorDataset(X_train, y_train)
        test_dataset = TensorDataset(X_test, y_test)
        
        train_loader = DataLoader(
            train_dataset, batch_size=config['batch_size'], 
            shuffle=True, num_workers=2, pin_memory=True
        )
        test_loader = DataLoader(
            test_dataset, batch_size=config['batch_size'], 
            shuffle=False, num_workers=2, pin_memory=True
        )
        
        # Create model with adaptive architecture and dynamic num_classes
        model = AdaptiveEmotionModel(input_shape, num_classes=num_classes, dropout_rate=config['dropout_rate'])
        
        # Print model info
        total_params = sum(p.numel() for p in model.parameters())
        print(f"Model parameters: {total_params:,}")
        
        # Train
        model, best_acc, best_f1, highest_acc, highest_f1, highest_epoch, history = train_adaptive_model(
            model, train_loader, test_loader, device, feature_key, epochs=config['epochs']
        )
        
        # Final evaluation
        model.eval()
        final_preds = []
        final_labels = []
        
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = outputs.max(1)
                
                final_preds.extend(predicted.cpu().numpy())
                final_labels.extend(labels.cpu().numpy())
        
        final_acc = accuracy_score(final_labels, final_preds)
        final_f1 = f1_score(final_labels, final_preds, average='weighted')
        
        results[exp_name] = {
            'best_test_acc': best_acc,
            'best_test_f1': best_f1,
            'highest_test_acc': highest_acc,
            'highest_test_f1': highest_f1,
            'highest_epoch': highest_epoch,
            'final_acc': final_acc,
            'final_f1': final_f1,
            'predictions': final_preds,
            'true_labels': final_labels,
            'history': history,
            'config': config,
            'architecture': config['architecture'],
            'total_params': total_params
        }
        
        # Analyze feature importance for DNN models with gating
        if feature_key != 'mel_spectrogram':
            # Get appropriate feature names
            if feature_key == 'mfcc' and 'mfcc_names' in features_dict:
                feature_names = features_dict['mfcc_names']
            elif feature_key == 'bangla_handcrafted' and 'bangla_names' in features_dict:
                feature_names = features_dict['bangla_names']
            elif feature_key == 'all_acoustic' and 'all_names' in features_dict:
                feature_names = features_dict['all_names']
            else:
                # Fallback to generic names if feature names not available
                feature_names = [f'feature_{i}' for i in range(features.shape[1])]
            
            # Make sure we have the right number of feature names
            if len(feature_names) != features.shape[1]:
                feature_names = [f'feature_{i}' for i in range(features.shape[1])]
            
            importance = analyze_feature_importance(model, test_loader, feature_names, device)
            if importance:
                results[exp_name]['feature_importance'] = importance
                print(f"\nTop 10 Most Important Features (by gate values):")
                for i, (feat_name, gate_val) in enumerate(importance[:10], 1):
                    print(f"  {i:2d}. {feat_name[:30]:30s} : {gate_val:.4f}")
        
        print(f"\nFinal Results:")
        print(f"  Best: Acc={final_acc*100:.2f}%, F1={final_f1:.4f}")
        print(f"  Highest: Acc={highest_acc:.2f}%, F1={highest_f1:.4f}")
        
        # Classification report with dynamic emotion names
        print("\nClassification Report:")
        # Ensure emotion_names matches the number of unique classes in predictions
        unique_preds = np.unique(np.concatenate([final_labels, final_preds]))
        report_names = [emotion_names[i] if i < len(emotion_names) else f'Class_{i}' 
                       for i in unique_preds]
        print(classification_report(final_labels, final_preds, target_names=report_names, labels=unique_preds))
    
    return results

# ==================== Main ====================

def main(cache_path=None, dataset_name='SUBESCO'):
    """
    Main execution
    
    Args:
        cache_path: Path to the features pickle file. If None, tries default paths
        dataset_name: Name of the dataset (for display purposes)
    """
    
    print("="*70)
    print(f"BANGLA SER: Adaptive Architecture with Feature Gating")
    print(f"Dataset: {dataset_name}")
    print("CNN+LSTM for Spectrograms | DNN+Gating for Static Features")
    print("="*70)
    
    # Handle different cache paths
    if cache_path is None:
        # Try default paths
        possible_paths = [
            '/kaggle/input/subesco-handcrafted-features/comprehensive_features.pkl',
            'comprehensive_features.pkl',
            f'{dataset_name.lower()}_features.pkl'
        ]
        
        for path in possible_paths:
            if Path(path).exists():
                cache_path = path
                break
        
        if cache_path is None:
            print("Error: No feature cache file found. Please provide cache_path parameter.")
            return None
    
    if not Path(cache_path).exists():
        print(f"Error: Feature cache not found at {cache_path}")
        return None
    
    # Load features
    features = prepare_features_from_cache(cache_path)
    
    # Get dataset info
    num_classes = features['num_classes']
    emotion_names = features.get('emotion_names', [f'Emotion_{i}' for i in range(num_classes)])
    
    print(f"\nDataset Statistics:")
    print(f"  Total samples: {len(features['labels'])}")
    print(f"  Number of classes: {num_classes}")
    print(f"  Class distribution:")
    unique, counts = np.unique(features['labels'], return_counts=True)
    for label, count, name in zip(unique, counts, emotion_names[:len(unique)]):
        print(f"    {name}: {count} samples ({count/len(features['labels'])*100:.1f}%)")
    
    # Show configurations
    print("\n" + "="*70)
    print("ARCHITECTURE ASSIGNMENTS:")
    print("="*70)
    
    configs_to_show = {
        'All Features': 'all_acoustic',
        'MFCC': 'mfcc',
        'Bangla Handcrafted': 'bangla_handcrafted',
        'Mel-Spectrogram': 'mel_spectrogram'
    }
    
    for name, key in configs_to_show.items():
        config = get_training_config(key)
        print(f"\n{name}:")
        print(f"  Architecture: {config['architecture']}")
        print(f"  Learning Rate: {config['learning_rate']:.0e}")
        print(f"  Batch Size: {config['batch_size']}")
        if 'Gating' in config['architecture']:
            print(f"  Feature Gating: ENABLED (learns feature importance)")
    
    # Run experiments
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f"\nDevice: {device}")
    if device == 'cuda':
        print(f"GPU: {torch.cuda.get_device_name(0)}")
    
    results = run_experiments(features, device, dataset_name)
    
    # Visualize feature importance if available
    print("\n" + "="*70)
    print("FEATURE IMPORTANCE ANALYSIS")
    print("="*70)
    visualize_feature_importance(results)
    
    # Summary
    print("\n" + "="*70)
    print(f"RESULTS SUMMARY - {dataset_name}")
    print("="*70)
    
    sorted_results = sorted(results.items(), key=lambda x: x[1]['highest_test_acc'], reverse=True)
    
    print(f"\n{'Rank':<4} {'Feature Type':<20} {'Architecture':<12} {'Params':<10} {'Final Acc':<10} {'Highest Acc':<12}")
    print("-" * 80)
    
    for rank, (name, metrics) in enumerate(sorted_results, 1):
        print(f"{rank:<4} {name:<20} {metrics['architecture']:<12} "
              f"{metrics['total_params']/1e6:.1f}M      "
              f"{metrics['final_acc']*100:>6.2f}%    "
              f"{metrics['highest_test_acc']:>8.2f}%")
    
    # Improvements
    if 'Mel-Spectrogram' in results:
        baseline = results['Mel-Spectrogram']['highest_test_acc']
        
        print("\n" + "="*70)
        print("IMPROVEMENTS OVER MEL-SPECTROGRAM:")
        print("="*70)
        
        for name in results:
            if name != 'Mel-Spectrogram':
                improvement = results[name]['highest_test_acc'] - baseline
                symbol = "✓" if improvement > 0 else "✗"
                print(f"{name}: {improvement:+.2f}% {symbol}")
    
    # Save results for this dataset
    results_file = f'{dataset_name.lower()}_results.pkl'
    with open(results_file, 'wb') as f:
        pickle.dump(results, f)
    print(f"\nResults saved to {results_file}")
    
    return results

# Convenience functions for specific datasets
def run_subesco():
    """Run experiments on SUBESCO dataset (7 emotions)"""
    return main(
        cache_path='/kaggle/input/subesco-handcrafted-features/comprehensive_features.pkl',
        dataset_name='SUBESCO'
    )

def run_banglaser():
    """Run experiments on BanglaSER dataset (5 emotions)"""
    return main(
        cache_path='/kaggle/input/banglaser-kbes/banglaser-kbes/banglaser_features.pkl',  # Update with actual path
        dataset_name='BanglaSER'
    )

def run_kbes():
    """Run experiments on KBES dataset (9 emotions)"""
    return main(
        cache_path='/kaggle/input/banglaser-kbes/banglaser-kbes/kbes_features.pkl',  # Update with actual path
        dataset_name='KBES'
    )

if __name__ == "__main__":
    # Default to SUBESCO, but can be easily changed
    # import sys
    
    # if len(sys.argv) > 1:
    #     dataset = sys.argv[1].lower()
    #     if dataset == 'banglaser':
    #         results = run_banglaser()
    #     elif dataset == 'kbes':
    #         results = run_kbes()
    #     else:
    #         results = run_subesco()
    # else:
    #     # Default behavior
    #     results = main()  # Will try to auto-detect or use SUBESCO

    run_subesco()

    run_banglaser()
    run_kbes()

BANGLA SER: Adaptive Architecture with Feature Gating
Dataset: SUBESCO
CNN+LSTM for Spectrograms | DNN+Gating for Static Features
Loading features from /kaggle/input/subesco-handcrafted-features/comprehensive_features.pkl
Mel-spectrogram shape: (7000, 3, 128, 94)
MFCC features shape: (7000, 80)
Bangla handcrafted features shape: (7000, 51)
All acoustic features shape: (7000, 131)
Total samples: 7000
Number of emotion classes: 7
Unique emotion labels: [0 1 2 3 4 5 6]
Emotion names: ['ANGRY', 'DISGUST', 'FEAR', 'HAPPY', 'NEUTRAL', 'SAD', 'SURPRISE']...

Dataset Statistics:
  Total samples: 7000
  Number of classes: 7
  Class distribution:
    ANGRY: 1000 samples (14.3%)
    DISGUST: 1000 samples (14.3%)
    FEAR: 1000 samples (14.3%)
    HAPPY: 1000 samples (14.3%)
    NEUTRAL: 1000 samples (14.3%)
    SAD: 1000 samples (14.3%)
    SURPRISE: 1000 samples (14.3%)

ARCHITECTURE ASSIGNMENTS:

All Features:
  Architecture: HybridFusion
  Learning Rate: 5e-04
  Batch Size: 16

MFCC:
  Archite

Epoch 1: 100%|██████████| 350/350 [00:06<00:00, 54.09it/s, loss=1.92, acc=20.9]


Epoch 1: Train Loss=2.1455, Train Acc=20.89%, Test Acc=33.50%, Test F1=0.2914


Epoch 2: 100%|██████████| 350/350 [00:05<00:00, 61.58it/s, loss=1.5, acc=30.6]


Epoch 2: Train Loss=1.7203, Train Acc=30.64%, Test Acc=48.14%, Test F1=0.4494


...

Epoch 122: Train Loss=0.0532, Train Acc=99.58%, Test Acc=57.22%, Test F1=0.5782


Epoch 123: 100%|██████████| 45/45 [00:01<00:00, 23.54it/s, loss=0.0911, acc=100]


Epoch 123: Train Loss=0.0477, Train Acc=100.00%, Test Acc=57.22%, Test F1=0.5784


Epoch 124: 100%|██████████| 45/45 [00:01<00:00, 23.39it/s, loss=0.0476, acc=99.9]


Epoch 124: Train Loss=0.0478, Train Acc=99.86%, Test Acc=57.78%, Test F1=0.5817
Early stopping at epoch 124

Training Summary:
  Best Model: Epoch 74, Acc=58.33%, F1=0.5890
  Highest: Epoch 74, Acc=58.33%, F1=0.5890

Final Results:
  Best: Acc=57.78%, F1=0.5817
  Highest: Acc=58.33%, F1=0.5890

Classification Report:
                precision    recall  f1-score   support

  Angry (High)       0.60      0.45      0.51        20
   Angry (Low)       0.35      0.45      0.39        20
Disgust (High)       0.56      0.45      0.50        20
 Disgust (Low)       0.56      0.45      0.50        20
  Happy (High)       0.90      0.90      0.90        20
   Happy (Low)       0.28      0.35      0.31        20
       Neutral       0.65      0.65      0.65        20
    Sad (High)       0.85      0.85      0.85        20
     Sad (Low)       0.59      0.65      0.62        20

      accuracy                           0.58       180
     macro avg       0.59      0.58      0.58       180
  weigh